**Работа с сетями в docker**

Реализуется через API network.

# `ls`

Выведет список доступных сетей.

In [1]:
%%bash
docker network ls

NETWORK ID     NAME      DRIVER    SCOPE
cac8977da21a   bridge    bridge    local
f8b2503d0640   host      host      local
b1d7e6bda275   none      null      local


# Подключение к сети

### При создании контейнера

Производится, через указывание опции `--net=<имя/id сети>`. Далее в примере демонстрирую, что как минимум ничего не сломалось.

In [5]:
%%bash
docker run --rm -di --net=none --name just_net ubuntu_curl
docker stop just_net

ee6e4f29315a7fcf3a5ad2ee39cdb86bb178ebb296563fee73b432a30799a33c
just_net


### Подключение/откллючение уже созданного контейнера

Производится с помощью комманды 
 - Для подключения используется `docker network connect <сеть> <контейнер>`;
 - Для отключения используется `docker network disconnect <сеть> <контейнер>`.

In [6]:
%%bash
# запускаю контейнер с nginx он по умолчанию
# прикрепляется к bridge
docker run --rm --name test_nginx -itd nginx
# создаю новую сетку и цепляю не нее этот nginx
docker network create test_network
sleep 10
docker network connect test_network test_nginx

# отцепляю nginx от bridge
docker network disconnect bridge test_nginx

docker stop test_nginx
docker network rm test_network

1ced62443c8437cdc2e4e3910957b2632cded2d709634543a6a212d6f4d08d7a
a8f26f0c7bea903a0c61a2d553b2ace6b96a952f1e0120159e1042463208a6f3
test_nginx
test_network


# Сети по умолчанию

В docker по умолчанию доступно 3 сети:

In [2]:
%%bash
docker network ls

NETWORK ID     NAME      DRIVER    SCOPE
cac8977da21a   bridge    bridge    local
f8b2503d0640   host      host      local
b1d7e6bda275   none      null      local


### Сеть none

Производит полное блокирование контейнера от сети.

Так попробуем запустить clickhouse на этой сети и сделать к нему запрос. `curl: (7) Failed to connect to localhost port 8123 after 0 ms: Connection refused` свидетельствует о том, что по заданному адресу ничего не отвечат.

In [5]:
%%bash
docker run --rm -d --name none_example -p 8123:8123 --net=none yandex/clickhouse-server
# нельзя забывать, что clickhouse долговато поднимается
sleep 10
curl localhost:8123
docker stop none_example

05db6909cc4de9e808ffe927ad06594fafce9ea6e0a54e6fadf38f04e266759b


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to localhost port 8123 after 0 ms: Connection refused


none_example


Так же из нутри контейнера, нельзя будет достучаться в интернет. Так в следующем примере я запускаю ubuntu с предустановленным curl и получаю, и не могу из него достучаться до google.com. `curl: (6) Could not resolve host: google.com` свидетельсвует о том, что из контейнера не получается достучаться до google.com.

In [2]:
%%bash
docker run -itd --rm --net=none --name none_example ubuntu_curl
docker exec none_example curl google.com
docker stop none_example

3dd6e8e5f072ff86380acf7f092093bb96a1a1d7a3b091aaaf32d6ee21763e3c


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: google.com


none_example


### Сеть host

Полностью убирает сетевую изоляцию контейнера, то есть контейнер делит с хостом сеть полностью.

Так в примере я запускаю yandex/clickhouse-server без указания каких-либо портов. В результате, этот clickhouse всеравно будет отвечать (даже без указанных портов).

In [6]:
%%bash
docker run --rm -d --name host_example --net=host yandex/clickhouse-server
# нельзя забывать, что clickhouse долговато поднимается
sleep 10
curl localhost:8123
docker stop host_example

69120ad84831e74d88b6085e5b38ca8ba2a0f05ff9b66ac2748a01b7974707e6


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     4    0     4    0     0   4149      0 --:--:-- --:--:-- --:--:--  4000


Ok.
host_example


### Сеть bridge

Сеть в которой контейнеры создаются по умолчанию. Далее показано несколько трюков.

**Весь этот раздел воспроиводится особым образом**

Я предварительно поднял контейнеры с yandex/clickhouse-server и ubuntu с предустановленным curl:

In [7]:
%%bash
docker ps

CONTAINER ID   IMAGE                      COMMAND            CREATED              STATUS              PORTS                          NAMES
3c653442fea3   ubuntu_curl                "/bin/bash"        About a minute ago   Up About a minute                                  ubu
3f047e11c417   yandex/clickhouse-server   "/entrypoint.sh"   2 minutes ago        Up 2 minutes        8123/tcp, 9000/tcp, 9009/tcp   click


Теперь демонстрирую результат комманды `insperct` (позволяет получить низкоуровневую инофрмацию о любом объекте docker) для сети bridge.

In [8]:
%%bash
docker inspect bridge

[
    {
        "Name": "bridge",
        "Id": "243b737447851abe893a50fab43e6fd6304423b81f848e31ae974692bb2388a5",
        "Created": "2023-03-22T16:23:03.478736367+03:00",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": [
                {
                    "Subnet": "172.17.0.0/16",
                    "Gateway": "172.17.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "3c653442fea36887a8ff860b337148053ba5583c74ad635eeaa0bce4f75c8432": {
                "Name": "ubu",
                "EndpointID": "f29acdda0cc093af06a9797386eed25d491bb67e9e09112bae235657c81f93c9",
                "MacAddress": "02:42:ac:11:00:03",
                "IPv4Address"

Обращаю внимание на поле "Containers" а в нем на поля "IPv4Address". По этим адресам мы можем общаться между контейнерами. Так вот например и из ubuntu спрошу clikchouse. Ok свидетельсвует о том, что я дейвительно дотянулся.

In [9]:
%%bash
docker exec ubu curl 172.17.0.2:8123

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     4    0     4    0     0   4944      0 --:--:-- --:--:-- --:--:--  4000


Ok.


Более того, несмотря на то, что я не указывал портов при поднятии контейнера clickhouse, через этот ip я могу достучаться до clickhouse и из хоста. В следующем примере, я сравниваю результат запроса на localhost и на этот ip.

- В первом случае никуда я не достучался и закономерно получил `curl: (7) Failed to connect to localhost port 8123 after 0 ms: Connection refused`;
- Во втором случа clickhouse мне отвечает как ни в чем не бывало `Ok.`.

In [10]:
%%bash
curl localhost:8123
curl 172.17.0.2:8123

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to localhost port 8123 after 0 ms: Connection refused
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     4    0     4    0     0   9302      0 --:--:-- --:--:-- --:--:--  4000


Ok.


# Сетевые интерфейсы

Каждый контейнер создает новый сетевой интерфейс. Сетевые интерфейсы в linux можно посмотреть с помощью утилиты `ifconfig`. Далее пример, как выболняется `ifconfig` по умолчанию (с установленным docker).

In [1]:
%%bash
ifconfig

docker0: flags=4099<UP,BROADCAST,MULTICAST>  mtu 1500
        inet 172.17.0.1  netmask 255.255.0.0  broadcast 172.17.255.255
        inet6 fe80::42:1eff:fe03:26e4  prefixlen 64  scopeid 0x20<link>
        ether 02:42:1e:03:26:e4  txqueuelen 0  (Ethernet)
        RX packets 32158  bytes 1793702 (1.7 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 39205  bytes 192903770 (192.9 MB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        inet6 ::1  prefixlen 128  scopeid 0x10<host>
        loop  txqueuelen 1000  (Local Loopback)
        RX packets 27239  bytes 6082445 (6.0 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 27239  bytes 6082445 (6.0 MB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

wlp0s20f3: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 192.168.100.10  netmask 255.255.255.0  b

А теперь запустим какой-нибудь контейнер и посмотрим результат `ifconfig`. Появляется `veth...`, который и представляет собой интерфейс этого контейнера.

In [2]:
%%bash
docker run --rm -itd --name ubuntu_inter ubuntu
ifconfig
docker stop ubuntu_inter

64034d23001182e6a73b5fc24ad5de41800b6fb7a043b72de0d17aa4cde44aa2
docker0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 172.17.0.1  netmask 255.255.0.0  broadcast 172.17.255.255
        inet6 fe80::42:1eff:fe03:26e4  prefixlen 64  scopeid 0x20<link>
        ether 02:42:1e:03:26:e4  txqueuelen 0  (Ethernet)
        RX packets 32158  bytes 1793702 (1.7 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 39206  bytes 192903900 (192.9 MB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        inet6 ::1  prefixlen 128  scopeid 0x10<host>
        loop  txqueuelen 1000  (Local Loopback)
        RX packets 29201  bytes 6753989 (6.7 MB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 29201  bytes 6753989 (6.7 MB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

veth29fa8ed: flags=4163<UP,BROADCAST,RUNNING

*Примечание* интерфейсы можно слушать с помощью утилиты `tcpdump`.

# Кастомные сети

Для создания собсвенной сети в api `network` используется команда `create`. Она создается используя DRIVER bridge, соответсвенно обладает всеми свойсвами сети bridge.

Для удаления сети используется команда `rm`, так в следующем примере будет создана сеть `new_net` и сразу же удалена.

In [2]:
%%bash
docker network create new_net
docker network ls
docker network rm new_netb

5222bef596010f96790439ddc3dd6a392850f90bb660c820e993c7329e653b89
NETWORK ID     NAME      DRIVER    SCOPE
e2df331917d9   bridge    bridge    local
f8b2503d0640   host      host      local
5222bef59601   new_net   bridge    local
b1d7e6bda275   none      null      local
new_net


# Пример изоляции

Продемонстрируем, что контейнеры развернутые в одной сети типа "bridge" могут достучаться друг до друга. А в разных - нет.

В следующей ячейке я создаю три контейнера - два из них в сети "bridge" и последний в кастомной сети "new_net".

In [25]:
%%bash
docker run --rm -itd --name ubuntu_bridge1 ubuntu_curl
docker run --rm -itd --name ubuntu_bridge2 ubuntu_curl

docker network create new_net
docker run --rm -itd --name ubuntu_new_net --net=new_net ubuntu_curl

268c559a31fa4de87f1f96e344429086ecaaf57bfced8d3872e464ed7393ac99
f6f2903f851b129dc98a5526c7aa52c5b4a51572088e1571b8361bdf20b1212d
b96ad8e7eaa1294d4debd7afc69ee180f76ee96df2eaea4ead6b66b318342736
f13eaef8c1e69a14c8529ba3614422fef68b76640a2c9b9a2af31a1f47f5c186


Получаю информацию о сетях. Тут самое главное это ip контейнеров:
- "ubuntu_bridge1" - 172.17.0.2;
- "ubuntu_bridge2" - 172.17.0.3;
- "ubuntu_new_net" - 172.21.0.2 (этот ip от запуска к запуску может меняться);

In [26]:
%%bash
docker inspect bridge
docker inspect new_net

[
    {
        "Name": "bridge",
        "Id": "f2435156fe9a8b2feb702177a4e2b67d89ab065fc643bdf8260e152c19342bfb",
        "Created": "2023-03-26T20:14:42.936227946+03:00",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": [
                {
                    "Subnet": "172.17.0.0/16",
                    "Gateway": "172.17.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "268c559a31fa4de87f1f96e344429086ecaaf57bfced8d3872e464ed7393ac99": {
                "Name": "ubuntu_bridge1",
                "EndpointID": "d67e3d4fb30ebbdfb8f60edb65598bf3b2dd72f558847c65fdb4ccef8c043989",
                "MacAddress": "02:42:ac:11:00:02",
                "I

Из контейнера "ubuntu_bridge1" стучусь в контейнер "ubuntu_bridge2" с которым они в одной подсети, и, конечно, получаю ответ.

In [27]:
%%bash
docker exec ubuntu_bridge1 ping -c 4 172.17.0.3

PING 172.17.0.3 (172.17.0.3) 56(84) bytes of data.
64 bytes from 172.17.0.3: icmp_seq=1 ttl=64 time=0.086 ms
64 bytes from 172.17.0.3: icmp_seq=2 ttl=64 time=0.109 ms
64 bytes from 172.17.0.3: icmp_seq=3 ttl=64 time=0.110 ms
64 bytes from 172.17.0.3: icmp_seq=4 ttl=64 time=0.083 ms

--- 172.17.0.3 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3078ms
rtt min/avg/max/mdev = 0.083/0.097/0.110/0.012 ms


Теперь из контейнера "ubuntu_bridge1" стучусь в контейнер "ubuntu_new_net". Они в разных подсетях потому и ответа я не получил.

In [31]:
%%bash
# echo в конце надо чтобы не выдавалась ошибка
# которую генерит ping когда у него не получилось
# достучаться до сервака
docker exec ubuntu_bridge1 ping -c 4 -q 172.21.0.2;  echo $?

PING 172.21.0.2 (172.21.0.2) 56(84) bytes of data.

--- 172.21.0.2 ping statistics ---
4 packets transmitted, 0 received, 100% packet loss, time 3062ms

1


В конце концов, все удаляю, чтобы не захломляться.

In [32]:
%%bash
docker stop ubuntu_bridge1 ubuntu_bridge2 ubuntu_new_net
docker network rm new_net

ubuntu_bridge1
ubuntu_bridge2
ubuntu_new_net
new_net
